In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [2]:
df=pd.read_csv("stud.csv")

In [3]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
x=df.drop(columns=['math_score'])
y=df['math_score']

In [5]:
num_features=x.select_dtypes(exclude='object').columns
cat_features=x.select_dtypes(include='object').columns

In [6]:
num_features

Index(['reading_score', 'writing_score'], dtype='object')

In [7]:
cat_features

Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course'],
      dtype='object')

In [8]:
x.head(3)

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93


In [18]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
num_transform=StandardScaler()
oh_transfomer=OneHotEncoder()
preprocessor=ColumnTransformer(
    [
        ("OneHotEncoder",oh_transfomer,cat_features),
        ('StandardScaler',num_transform,num_features)
    ]
)

In [27]:
x=preprocessor.fit_transform(x)

In [28]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train.shape,x_test.shape

((800, 19), (200, 19))

In [29]:
x_train

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.05694554,  0.45733301],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.94779033,  0.98406266],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.35894946,  1.18158627],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
        -0.49126664, -0.99117351],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
        -1.45063795, -0.99117351],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.4960025 ,  1.37910989]], shape=(800, 19))

evaluation part


In [42]:
from sklearn.metrics import mean_absolute_error


def evaluate_model(true,predict):
    mae=mean_absolute_error(true,predict)
    mse=mean_squared_error(true,predict)
    rmse=np.sqrt(mean_squared_error(true,predict))
    r2score=r2_score(true,predict)
    return mae,mse,rmse,r2score

In [31]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor

In [43]:
models={
    "LinearRegression":LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "K-Neighbours Regression":KNeighborsRegressor(),
    "DecisionTreeRegressor":DecisionTreeRegressor(),
    "RandomForestRegressor":RandomForestRegressor(),
    "XGBRegressor":XGBRegressor(),
    "CatBoostRegressor":CatBoostRegressor(verbose=False),
    "AdaBoostRegressor":AdaBoostRegressor()
}

model_list=[]
r2_list=[]
for model_name, model in models.items():
    model.fit(x_train,y_train)
    
    y_pred=model.predict(x_test)
    
    model_test_mae,model_test_mse,model_train_rmse,model_r2=evaluate_model(y_test,y_pred)
    print(f"Model: {model_name}")
    print(f"MAE: {model_test_mae:.2f}")
    print(f"MSE: {model_test_mse:.2f}")
    print(f"RMSE: {model_train_rmse:.2f}")
    print(f"R2 Score: {model_r2:.2f}")
    print('..............................')
    model_list.append(model)
    r2_list.append(model_r2)
    print('...................................')


Model: LinearRegression
MAE: 4.21
MSE: 29.10
RMSE: 5.39
R2 Score: 0.88
..............................
...................................
Model: Lasso
MAE: 5.16
MSE: 42.51
RMSE: 6.52
R2 Score: 0.83
..............................
...................................
Model: Ridge
MAE: 4.21
MSE: 29.06
RMSE: 5.39
R2 Score: 0.88
..............................
...................................
Model: K-Neighbours Regression
MAE: 5.64
MSE: 52.68
RMSE: 7.26
R2 Score: 0.78
..............................
...................................
Model: DecisionTreeRegressor
MAE: 6.05
MSE: 57.78
RMSE: 7.60
R2 Score: 0.76
..............................
...................................
Model: RandomForestRegressor
MAE: 4.72
MSE: 36.91
RMSE: 6.08
R2 Score: 0.85
..............................
...................................
Model: XGBRegressor
MAE: 5.06
MSE: 41.90
RMSE: 6.47
R2 Score: 0.83
..............................
...................................
Model: RandomForestRegressor
MAE: 4.72
MSE:

In [44]:
r2_list

[0.8804332983749565,
 0.8253197323627853,
 0.8805931485028737,
 0.7834973847119895,
 0.762552889309702,
 0.8483317586229712,
 0.8277965784072876,
 0.8516318920747058,
 0.845007720193482]

In [46]:
model_list

[LinearRegression(),
 Lasso(),
 Ridge(),
 KNeighborsRegressor(),
 DecisionTreeRegressor(),
 RandomForestRegressor(),
 XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...),
 AdaBoostRegressor()]

In [49]:
results_df = pd.DataFrame({
    'Model': list(models.keys()),
    'R2_Score': r2_list
})
results_df

,Model,R2_Score
0,LinearRegression,0.880433
1,Lasso,0.825320
2,Ridge,0.880593
3,K-Neighbours Regression,0.783497
4,DecisionTreeRegressor,0.762553
5,RandomForestRegressor,0.848332
6,XGBRegressor,0.827797
7,CatBoostRegressor,0.851632
8,AdaBoostRegressor,0.845008


linear_regression is working well here